# Module 09: Ethics, IRB, and GDPR Compliance

**Difficulty**: ⭐⭐⭐ (Advanced)

**Estimated Time**: 90 minutes

**Prerequisites**: Module 08 - Research Design and Validity

## Learning Objectives

By the end of this notebook, you will be able to:

1. Apply five core ethical principles (fairness, transparency, accountability, privacy, beneficence) to research design
2. Determine when Institutional Review Board (IRB) review is required and navigate exemption categories
3. Ensure GDPR compliance for data processing involving European data subjects
4. Detect algorithmic bias using fairness metrics and implement mitigation strategies
5. Implement privacy by design principles in research and ML systems

## Setup

Let’s import the libraries we’ll use in this notebook.

In [ ]:
# Standard data science libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from enum import Enum
from typing import Dict, List, Tuple
import hashlib
import json

# Configuration for better visualizations
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Set random seeds for reproducibility
np.random.seed(42)

print("✓ Libraries imported successfully!")
print(f"✓ Notebook created: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1. Five Core Ethical Principles in Research

Modern research ethics rests on five foundational principles that guide how we conduct research responsibly. These principles emerged from centuries of ethical thought, from the Hippocratic Oath to the Nuremberg Code and contemporary frameworks like the EU Ethics Guidelines.

### Principle 1: Fairness

**Definition**: Avoid bias and discrimination; ensure benefits and burdens are distributed equitably across all population groups.

**In Practice**:
- Regularly audit models for disparate impact across demographic groups
- Consider multiple definitions of fairness (they often conflict)
- Include diverse representation in data collection and testing
- Acknowledge that perfect fairness is impossible; choose fairness metrics intentionally

**Example**: A hiring algorithm must be tested for equal performance across gender and race. If it has 95% accuracy for men but 78% for women, it violates fairness even if overall accuracy is high.

### Principle 2: Transparency

**Definition**: Make research methods and decision-making processes explainable and understandable to stakeholders.

**In Practice**:
- Document model decisions with LIME, SHAP, or similar explanations
- Clearly state limitations and assumptions
- Provide plain-language summaries alongside technical details
- Enable external auditing and review

**Example**: A loan approval system should explain to applicants WHY their application was rejected, not just that it was.

### Principle 3: Accountability

**Definition**: Clearly assign responsibility for research decisions and impacts; conduct and document impact assessments.

**In Practice**:
- Designate clear roles: who decides, who implements, who verifies?
- Conduct Data Protection Impact Assessments (DPIA)
- Maintain audit trails of all decisions and changes
- Plan for remediation if harms are discovered

**Example**: AI/ML teams must document who tested for bias, who signed off on deployment, and how they will respond if bias is discovered post-deployment.

### Principle 4: Privacy

**Definition**: Protect personal data through encryption, access controls, and purpose limitation.

**In Practice**:
- Encrypt data at rest and in transit
- Implement role-based access controls
- De-identify data when possible
- Limit data use to stated purposes

**Example**: A medical research study collects health data for cardiovascular research. Using the same data to study genetic predispositions without explicit new consent violates privacy principles (purpose limitation).

### Principle 5: Beneficence

**Definition**: Maximize benefits while minimizing potential harms to research participants.

**In Practice**:
- Conduct risk-benefit analyses before research begins
- Design studies to answer important questions
- Implement safeguards to protect vulnerable populations
- Share results transparently

**Example**: A machine learning model deployed in healthcare must reduce diagnostic errors more than it creates new mistakes. The benefits must justify any harms from occasional errors.

In [ ]:
# Create an ethical principles framework class

class EthicalPrinciple:
    """Represents an ethical principle with assessment criteria."""
    
    def __init__(self, name: str, definition: str, key_questions: List[str]):
        self.name = name
        self.definition = definition
        self.key_questions = key_questions
    
    def assess(self, research_plan: Dict[str, str]) -> Dict[str, any]:
        """Assess a research plan against this principle."""
        return {
            'principle': self.name,
            'status': 'needs_review',
            'assessment_items': self.key_questions
        }

# Define the five ethical principles
principles = {
    'Fairness': EthicalPrinciple(
        name='Fairness',
        definition='Avoid bias and ensure equitable distribution of benefits/burdens',
        key_questions=[
            'Have we tested for disparate impact across demographic groups?',
            'Do protected groups have equal model performance?',
            'Have we considered trade-offs between different fairness metrics?',
            'Is representation diverse in training data?'
        ]
    ),
    'Transparency': EthicalPrinciple(
        name='Transparency',
        definition='Make research methods and decisions explainable',
        key_questions=[
            'Can we explain individual model predictions?',
            'Are limitations clearly documented?',
            'Can external parties audit our methods?',
            'Do stakeholders understand the approach in plain language?'
        ]
    ),
    'Accountability': EthicalPrinciple(
        name='Accountability',
        definition='Clearly assign responsibility for decisions and impacts',
        key_questions=[
            'Who is responsible for this research?',
            'Do we have audit trails of all key decisions?',
            'Have we conducted impact assessments?',
            'What is our plan for remediation if harms occur?'
        ]
    ),
    'Privacy': EthicalPrinciple(
        name='Privacy',
        definition='Protect personal data through technical and organizational measures',
        key_questions=[
            'Is data encrypted at rest and in transit?',
            'Are access controls role-based and minimal?',
            'Have we de-identified data where possible?',
            'Is data use limited to stated purposes?'
        ]
    ),
    'Beneficence': EthicalPrinciple(
        name='Beneficence',
        definition='Maximize benefits while minimizing harms',
        key_questions=[
            'Do benefits clearly outweigh risks?',
            'Have we protected vulnerable populations?',
            'Is the research question important enough to justify any risks?',
            'Will we share results transparently?'
        ]
    )
}

# Display the framework
print("\n" + "="*70)
print("FIVE CORE ETHICAL PRINCIPLES FRAMEWORK")
print("="*70)

for principle_name, principle in principles.items():
    print(f"\n{principle_name.upper()}")
    print("-" * 70)
    print(f"Definition: {principle.definition}")
    print(f"\nKey Assessment Questions:")
    for i, question in enumerate(principle.key_questions, 1):
        print(f"  {i}. {question}")

## 2. Institutional Review Board (IRB) Requirements

An Institutional Review Board (IRB) is a committee that reviews and approves research involving human subjects. Understanding when IRB review is required is crucial for research compliance.

### What Requires IRB Review?

**Research** that involves:
1. **Human subjects**: Any living person
2. **Interventions or interactions**: Deliberate manipulation, observation, or engagement
3. **Private identifiable information**: PII that could identify individuals
4. **Generalizable knowledge**: Results intended to be shared or published

**Example requiring review**: Survey of 100 customers about product preferences (involves subjects, interaction, and intent to publish)

**Example NOT requiring review**: Analysis of pre-existing anonymized dataset with no human contact

### Exemption Categories (45 CFR 46.101(b))

Some research can be **exempt** from full IRB review if it meets specific criteria:

| Category | Description | Typical Examples |
|----------|-------------|------------------|
| (1) | Educational settings; normal educational practices | Classroom surveys, teaching experiments |
| (2) | Educational tests, surveys, interviews (if anonymous) | Anonymous customer satisfaction surveys |
| (3) | Existing data analysis (secondary data) | Analysis of published datasets |
| (4) | Quality improvement projects | Internal process improvements |
| (5) | Public benefit/service programs | Evaluation of government programs |
| (6) | Taste/consumer acceptance studies | Food/beverage preference studies |
| (7) | Certain foreign research | International studies with local ethics approval |

### Levels of Review

**Exempt**: No IRB review required (but must be carefully evaluated)
- Low risk to subjects
- No sensitive data
- Minimal burden on subjects

**Expedited**: Streamlined review (45 days typical)
- Minimal risk projects
- Amendments to approved research
- Continuing review

**Full Board**: Complete committee review (typically 60+ days)
- More than minimal risk
- Vulnerable populations (children, prisoners, pregnant women)
- Sensitive topics

In [ ]:
# IRB Review Determination Tool

class IRBReviewDetermination:
    """Determines whether research requires IRB review and what level."""
    
    def __init__(self):
        self.criteria = {
            'human_subjects': {
                'weight': 2,
                'question': 'Does research involve living human subjects?',
                'required': True
            },
            'intervention_interaction': {
                'weight': 2,
                'question': 'Does it involve intervention, interaction, or manipulation?',
                'required_with': ['human_subjects']
            },
            'identifiable_data': {
                'weight': 2,
                'question': 'Does it use private identifiable information?',
                'required_with': ['human_subjects']
            },
            'generalizable_knowledge': {
                'weight': 1,
                'question': 'Is knowledge intended to be generalizable/published?',
                'required_with': ['human_subjects']
            },
            'minimal_risk': {
                'weight': 2,
                'question': 'Is risk to subjects no greater than everyday life?',
                'increases_exemption': True
            },
            'anonymous': {
                'weight': 2,
                'question': 'Are all data completely anonymous?',
                'increases_exemption': True
            }
        }
    
    def evaluate_research(self, research_info: Dict[str, bool]) -> Dict:
        """Evaluate a research project for IRB requirements."""
        
        # Check if IRB review needed
        needs_irb = (
            research_info.get('human_subjects', False) and
            (
                research_info.get('intervention_interaction', False) or
                research_info.get('identifiable_data', False)
            )
        )
        
        # Determine level if needed
        review_level = 'Not Required'
        if needs_irb:
            # Check for exemptions
            if (research_info.get('minimal_risk', True) and 
                research_info.get('anonymous', True)):
                review_level = 'Exempt (Category 3 - Secondary Data)'
            elif research_info.get('minimal_risk', False):
                review_level = 'Expedited Review'
            else:
                review_level = 'Full Board Review'
        
        return {
            'requires_irb': needs_irb,
            'review_level': review_level,
            'rationale': self._generate_rationale(research_info, needs_irb)
        }
    
    def _generate_rationale(self, info: Dict, needs_irb: bool) -> str:
        """Generate explanation for determination."""
        if needs_irb:
            if 'Exempt' in self._get_level(info):
                return 'Secondary analysis of de-identified data qualifies for exemption'
            elif 'Expedited' in self._get_level(info):
                return 'Low-risk research with minimal burden qualifies for expedited review'
            else:
                return 'Higher risk or vulnerable populations require full board review'
        return 'No human subject research involved; IRB review not required'
    
    def _get_level(self, info: Dict) -> str:
        if (info.get('minimal_risk', True) and info.get('anonymous', True)):
            return 'Exempt'
        elif info.get('minimal_risk', False):
            return 'Expedited'
        return 'Full Board'

# Initialize the tool
irb_tool = IRBReviewDetermination()

# Example: Evaluate different research scenarios
scenarios = {
    'Scenario A: Customer Survey': {
        'human_subjects': True,
        'intervention_interaction': True,  # Asking questions
        'identifiable_data': False,  # Anonymous survey
        'generalizable_knowledge': True,
        'minimal_risk': True,
        'anonymous': True
    },
    'Scenario B: Medical Trial': {
        'human_subjects': True,
        'intervention_interaction': True,
        'identifiable_data': True,
        'generalizable_knowledge': True,
        'minimal_risk': False,  # Medical intervention
        'anonymous': False
    },
    'Scenario C: Secondary Data Analysis': {
        'human_subjects': False,  # Only analyzing historical data
        'intervention_interaction': False,
        'identifiable_data': False,  # De-identified dataset
        'generalizable_knowledge': True,
        'minimal_risk': True,
        'anonymous': True
    }
}

print("\n" + "="*70)
print("IRB REVIEW DETERMINATION RESULTS")
print("="*70)

for scenario_name, scenario_data in scenarios.items():
    result = irb_tool.evaluate_research(scenario_data)
    print(f"\n{scenario_name}")
    print("-" * 70)
    print(f"Requires IRB Review: {result['requires_irb']}")
    print(f"Review Level: {result['review_level']}")
    print(f"Rationale: {result['rationale']}")

## 3. GDPR Essentials for Data Protection

The General Data Protection Regulation (GDPR) is EU law governing how personal data of EU residents must be handled. It applies globally to any organization processing data about European data subjects.

### When GDPR Applies

GDPR applies if:
- You collect data from **anyone in the EEA** (EU + Iceland, Liechtenstein, Norway)
- Data subject's location: NOT the location of your business
- Data is processed **anywhere** (cloud, servers in any country)

**Example**: A US company with no EU offices collecting data on German customers MUST comply with GDPR

### Key GDPR Principles (Article 5)

1. **Lawfulness, Fairness, Transparency**: Data processing must be legal, fair, and transparent
2. **Purpose Limitation**: Data collected for one purpose cannot be reused for another without new consent
3. **Data Minimization**: Collect only what's necessary
4. **Accuracy**: Keep data accurate and up-to-date
5. **Storage Limitation**: Don't keep data longer than needed
6. **Integrity & Confidentiality**: Protect data security
7. **Accountability**: Demonstrate compliance

### Valid Consent (Critical Requirement)

Consent under GDPR must be:

- **Freely Given**: No coercion; users can refuse
- **Specific**: For clearly defined purposes
- **Informed**: Users know what data and how it's used
- **Unambiguous**: Explicit opt-in (NOT pre-checked boxes)
- **Distinguishable**: Clear language separate from other terms
- **Revocable**: Users can withdraw anytime

**NOT Valid**:
- Pre-checked consent boxes ❌
- "Continue using our service" = consent ❌
- Bundled with other terms ❌
- Vague descriptions of use ❌

### Special Category Data

Data revealing:
- Race or ethnicity
- Political opinions
- Religious/philosophical beliefs
- Trade union membership
- Genetic data
- Biometric data (for identification)
- Health data
- Sex life or sexual orientation

**Rule**: These require **explicit consent** or narrow exceptions (medical, vital interests, etc.)

### Privacy by Design

Build privacy into systems from the start, not as an afterthought:

1. **Data Minimization**: Collect only necessary data
2. **Purpose Limitation**: Define and document use cases upfront
3. **Access Control**: Least privilege principle
4. **Encryption**: Data at rest and in transit
5. **De-identification**: Remove/hash PII when possible
6. **Retention Policies**: Delete data when no longer needed
7. **Audit Logging**: Track who accessed what and when

In [ ]:
# GDPR Compliance Implementation

class ConsentValidator:
    """Validates whether consent meets GDPR requirements."""
    
    def __init__(self):
        self.requirements = [
            'freely_given',
            'specific',
            'informed',
            'unambiguous',
            'distinguishable',
            'revocable'
        ]
    
    def validate_consent_text(self, consent_statement: str) -> Dict[str, any]:
        """Analyze consent text for GDPR compliance."""
        checks = {
            'freely_given': 'can refuse' in consent_statement.lower() or 
                            'optional' in consent_statement.lower(),
            'specific': 'purpose' in consent_statement.lower() or 
                       'for' in consent_statement.lower(),
            'informed': 'data' in consent_statement.lower() or 
                       'information' in consent_statement.lower(),
            'unambiguous': 'explicit' in consent_statement.lower() or 
                          'I agree' in consent_statement or
                          'opt-in' in consent_statement.lower(),
            'distinguishable': len(consent_statement) > 50,  # Separate substance
            'revocable': 'withdraw' in consent_statement.lower() or 
                        'revoke' in consent_statement.lower()
        }
        
        compliance_score = sum(checks.values()) / len(checks)
        
        return {
            'individual_checks': checks,
            'compliance_score': f"{compliance_score*100:.0f}%",
            'is_compliant': compliance_score >= 0.83,  # Need 5/6 minimum
            'feedback': self._generate_feedback(checks)
        }
    
    def _generate_feedback(self, checks: Dict[str, bool]) -> List[str]:
        """Generate improvement suggestions."""
        feedback = []
        if not checks['freely_given']:
            feedback.append("Add language indicating consent is optional")
        if not checks['specific']:
            feedback.append("Clearly specify the purpose of data collection")
        if not checks['informed']:
            feedback.append("Explain what data is collected and how it's used")
        if not checks['unambiguous']:
            feedback.append("Use explicit opt-in language (e.g., 'I agree to...')")
        if not checks['distinguishable']:
            feedback.append("Separate consent from other terms and conditions")
        if not checks['revocable']:
            feedback.append("Include instructions for withdrawing consent")
        return feedback if feedback else ["✓ Excellent - Meets GDPR requirements"]


class PrivacyByDesignFramework:
    """Implements privacy by design principles."""
    
    def __init__(self):
        self.principles = {
            'data_minimization': 'Collect only necessary data',
            'purpose_limitation': 'Define use cases explicitly',
            'access_control': 'Implement least privilege',
            'encryption': 'Encrypt data at rest and transit',
            'de_identification': 'Remove PII when possible',
            'retention_policy': 'Delete data when no longer needed',
            'audit_logging': 'Track all data access'
        }
    
    def create_privacy_policy(self, data_fields: List[str], 
                             purposes: List[str],
                             retention_days: int) -> Dict:
        """Generate privacy policy elements."""
        
        return {
            'collected_data': data_fields,
            'purposes': purposes,
            'retention_period': f"{retention_days} days",
            'deletion_policy': f"Data deleted after {retention_days} days of inactivity",
            'access_controls': "Role-based access control implemented",
            'encryption': "AES-256 encryption for sensitive fields",
            'audit_trail': "All access logged and monitored"
        }

# Test the validators
print("\n" + "="*70)
print("GDPR COMPLIANCE CHECKING")
print("="*70)

validator = ConsentValidator()

# Example 1: Good consent
good_consent = """
CONSENT FOR DATA COLLECTION

I understand and agree that my email address will be used to send you 
weekly newsletter updates about product features and promotions. I can 
withdraw this consent at any time by clicking the unsubscribe link in 
the email or contacting support@company.com. This consent is optional 
and will not affect my access to core services.
"""

print("\nExample 1: GOOD CONSENT TEXT")
print("-" * 70)
result_good = validator.validate_consent_text(good_consent)
print(f"Compliance Score: {result_good['compliance_score']}")
print(f"Is GDPR Compliant: {result_good['is_compliant']}")
print(f"\nChecks:")
for check, passed in result_good['individual_checks'].items():
    status = "✓" if passed else "✗"
    print(f"  {status} {check.replace('_', ' ').title()}")
if result_good['feedback']:
    print(f"\nFeedback: {result_good['feedback'][0]}")

# Example 2: Bad consent
bad_consent = "By using this website, you agree to our terms."

print("\n\nExample 2: POOR CONSENT TEXT")
print("-" * 70)
result_bad = validator.validate_consent_text(bad_consent)
print(f"Compliance Score: {result_bad['compliance_score']}")
print(f"Is GDPR Compliant: {result_bad['is_compliant']}")
print(f"\nChecks:")
for check, passed in result_bad['individual_checks'].items():
    status = "✓" if passed else "✗"
    print(f"  {status} {check.replace('_', ' ').title()}")
print(f"\nRequired Improvements:")
for i, feedback in enumerate(result_bad['feedback'], 1):
    print(f"  {i}. {feedback}")

## 4. Algorithmic Bias Detection and Mitigation

Bias in machine learning systems can perpetuate discrimination and violate ethical principles. Detecting and mitigating bias requires systematic approaches across three stages:

### Three Bias Mitigation Stages

#### Pre-processing (Before Model Training)
- **Goal**: Clean biased training data
- **Techniques**:
  - Oversampling underrepresented groups
  - Reweighting training samples
  - Removing biased features (redlining attributes)
  - Improving data representation

#### In-processing (During Model Training)
- **Goal**: Train model with fairness constraints
- **Techniques**:
  - Add fairness constraints to loss function
  - Use fair learning algorithms
  - Adversarial debiasing
  - Calibrated predictions

#### Post-processing (After Predictions)
- **Goal**: Adjust predictions to meet fairness criteria
- **Techniques**:
  - Threshold optimization per group
  - Equalized odds adjustment
  - Output post-processing

### Fairness Metrics

Different metrics capture different aspects of fairness:

| Metric | Definition | When to Use |
|--------|-----------|-------------|
| **Demographic Parity** | P(pred=1&#124;A=0) = P(pred=1&#124;A=1) | Hiring, lending (selection rates equal) |
| **Equalized Odds** | False positive and false negative rates equal across groups | Credit approval (minimize disparate impact) |
| **Predictive Parity** | P(Y=1&#124;pred=1,A=0) = P(Y=1&#124;pred=1,A=1) | Medical diagnosis (precision equal) |
| **Disparate Impact Ratio** | Selection rate minority / selection rate majority ≥ 0.8 | Legal standard (80% rule) |

**Key Insight**: These metrics often conflict. You must choose which fairness definition matters for your use case.

In [ ]:
# Fairness Metrics Implementation

class FairnessMetricsCalculator:
    """Calculates fairness metrics for ML models."""
    
    @staticmethod
    def demographic_parity(y_pred: np.ndarray, protected_attr: np.ndarray) -> Dict:
        """
        Calculate demographic parity (selection rates equal across groups).
        Ideal: ratio close to 1.0 (equal selection rates)
        """
        group_0_selection = (y_pred[protected_attr == 0] == 1).mean()
        group_1_selection = (y_pred[protected_attr == 1] == 1).mean()
        
        parity_ratio = group_0_selection / group_1_selection if group_1_selection > 0 else 0
        
        return {
            'metric': 'Demographic Parity',
            'group_0_selection_rate': f"{group_0_selection*100:.1f}%",
            'group_1_selection_rate': f"{group_1_selection*100:.1f}%",
            'parity_ratio': f"{parity_ratio:.3f}",
            'is_fair': 0.8 <= parity_ratio <= 1.25,  # 80% rule threshold
            'interpretation': f"Minority selected at {parity_ratio*100:.0f}% of majority rate"
        }
    
    @staticmethod
    def equalized_odds(y_true: np.ndarray, y_pred: np.ndarray, 
                      protected_attr: np.ndarray) -> Dict:
        """
        Calculate equalized odds (FPR and FNR equal across groups).
        Ideal: both rates equal between groups
        """
        # True positive rates
        tpr_group_0 = (y_pred[protected_attr == 0][y_true[protected_attr == 0] == 1] == 1).mean()
        tpr_group_1 = (y_pred[protected_attr == 1][y_true[protected_attr == 1] == 1] == 1).mean()
        
        # False positive rates  
        fpr_group_0 = (y_pred[protected_attr == 0][y_true[protected_attr == 0] == 0] == 1).mean()
        fpr_group_1 = (y_pred[protected_attr == 1][y_true[protected_attr == 1] == 0] == 1).mean()
        
        return {
            'metric': 'Equalized Odds',
            'tpr_difference': f"{abs(tpr_group_0 - tpr_group_1)*100:.1f}%",
            'fpr_difference': f"{abs(fpr_group_0 - fpr_group_1)*100:.1f}%",
            'tpr_group_0': f"{tpr_group_0*100:.1f}%",
            'tpr_group_1': f"{tpr_group_1*100:.1f}%",
            'fpr_group_0': f"{fpr_group_0*100:.1f}%",
            'fpr_group_1': f"{fpr_group_1*100:.1f}%"
        }

# Demonstrate with simulated data
print("\n" + "="*70)
print("BIAS DETECTION: FAIRNESS METRICS")
print("="*70)

np.random.seed(42)

# Create simulated hiring data
n_samples = 1000
protected_attr = np.random.binomial(1, 0.3, n_samples)  # 30% minority group

# Biased model: worse performance for protected group
y_true = np.random.binomial(1, 0.5, n_samples)
y_pred = np.zeros(n_samples)

# Group 0 (majority): 70% accuracy
y_pred[protected_attr == 0] = (np.random.random(sum(protected_attr == 0)) < 0.7).astype(int)

# Group 1 (minority): 55% accuracy (biased!)
y_pred[protected_attr == 1] = (np.random.random(sum(protected_attr == 1)) < 0.55).astype(int)

calculator = FairnessMetricsCalculator()

# Calculate metrics
dem_parity = calculator.demographic_parity(y_pred, protected_attr)
eq_odds = calculator.equalized_odds(y_true, y_pred, protected_attr)

print("\nDEMOGRAPHIC PARITY (Selection Rates)")
print("-" * 70)
for key, value in dem_parity.items():
    if key not in ['metric']:
        print(f"{key.replace('_', ' ').title()}: {value}")

print(f"\n⚠️ BIAS DETECTED: {dem_parity['group_0_selection_rate']} vs {dem_parity['group_1_selection_rate']}")
print(f"   Minority group selected at {(float(dem_parity['parity_ratio'])*100):.0f}% of majority rate")

print("\n\nEQUALIZED ODDS (Error Rates)")
print("-" * 70)
for key, value in eq_odds.items():
    if key not in ['metric']:
        print(f"{key.replace('_', ' ').title()}: {value}")

print(f"\n⚠️ TPR Difference: {eq_odds['tpr_difference']} (should be <5%)")
print(f"   FPR Difference: {eq_odds['fpr_difference']} (should be <5%)")

In [ ]:
# Visualize bias across groups

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Selection rates
groups = ['Majority Group\n(Protected=0)', 'Minority Group\n(Protected=1)']
selection_rates = [
    float(dem_parity['group_0_selection_rate'].rstrip('%')) / 100,
    float(dem_parity['group_1_selection_rate'].rstrip('%')) / 100
]
colors = ['#2ecc71', '#e74c3c']
axes[0, 0].bar(groups, selection_rates, color=colors, alpha=0.7, edgecolor='black')
axes[0, 0].axhline(y=0.5, color='blue', linestyle='--', linewidth=2, label='Equal (Fair)')
axes[0, 0].set_ylabel('Selection Rate', fontsize=11, fontweight='bold')
axes[0, 0].set_title('Demographic Parity: Selection Rates by Group', fontsize=12, fontweight='bold')
axes[0, 0].set_ylim([0, 1])
axes[0, 0].legend()
for i, rate in enumerate(selection_rates):
    axes[0, 0].text(i, rate + 0.03, f'{rate*100:.1f}%', ha='center', fontweight='bold')

# Plot 2: True positive rates (recall)
tpr_group_0 = float(eq_odds['tpr_group_0'].rstrip('%')) / 100
tpr_group_1 = float(eq_odds['tpr_group_1'].rstrip('%')) / 100
axes[0, 1].bar(groups, [tpr_group_0, tpr_group_1], color=colors, alpha=0.7, edgecolor='black')
axes[0, 1].axhline(y=0.5, color='blue', linestyle='--', linewidth=2, label='Equal (Fair)')
axes[0, 1].set_ylabel('True Positive Rate', fontsize=11, fontweight='bold')
axes[0, 1].set_title('Equalized Odds: True Positive Rates', fontsize=12, fontweight='bold')
axes[0, 1].set_ylim([0, 1])
axes[0, 1].legend()
for i, rate in enumerate([tpr_group_0, tpr_group_1]):
    axes[0, 1].text(i, rate + 0.03, f'{rate*100:.1f}%', ha='center', fontweight='bold')

# Plot 3: False positive rates (false alarm rate)
fpr_group_0 = float(eq_odds['fpr_group_0'].rstrip('%')) / 100
fpr_group_1 = float(eq_odds['fpr_group_1'].rstrip('%')) / 100
axes[1, 0].bar(groups, [fpr_group_0, fpr_group_1], color=colors, alpha=0.7, edgecolor='black')
axes[1, 0].axhline(y=0.0, color='blue', linestyle='--', linewidth=2, label='Equal (Fair)')
axes[1, 0].set_ylabel('False Positive Rate', fontsize=11, fontweight='bold')
axes[1, 0].set_title('Equalized Odds: False Positive Rates', fontsize=12, fontweight='bold')
axes[1, 0].set_ylim([0, 0.3])
axes[1, 0].legend()
for i, rate in enumerate([fpr_group_0, fpr_group_1]):
    axes[1, 0].text(i, rate + 0.01, f'{rate*100:.1f}%', ha='center', fontweight='bold')

# Plot 4: Fairness trade-offs (conceptual)
parity_ratio = float(dem_parity['parity_ratio'])
fairness_metrics = ['Demographic\nParity', 'Equalized\nOdds', 'Predictive\nParity']
fairness_scores = [parity_ratio if parity_ratio <= 1 else 2 - parity_ratio,  # Normalize
                  0.5,  # Simulated
                  0.6]  # Simulated

axes[1, 1].barh(fairness_metrics, fairness_scores, color=['#e74c3c', '#f39c12', '#e67e22'], alpha=0.7, edgecolor='black')
axes[1, 1].axvline(x=0.8, color='green', linestyle='--', linewidth=2, label='Acceptable (>0.8)')
axes[1, 1].set_xlabel('Fairness Score', fontsize=11, fontweight='bold')
axes[1, 1].set_title('Fairness Metric Trade-offs', fontsize=12, fontweight='bold')
axes[1, 1].set_xlim([0, 1])
axes[1, 1].legend()
for i, score in enumerate(fairness_scores):
    axes[1, 1].text(score + 0.02, i, f'{score:.2f}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 KEY INSIGHT: Fairness metrics often conflict!")
print("   Different definitions can't all be satisfied simultaneously.")
print("   You must choose which fairness definition is most important for your use case.")

## Exercise 1: Ethical Analysis of a Research Project

You are designing a machine learning system to predict employee turnover risk. The model will analyze employee data (age, salary, performance ratings, email sentiment) to identify who is likely to leave.

**Task**: Analyze this project against the five ethical principles.

For each principle, answer:
1. What are the potential ethical concerns?
2. What mitigation strategies would you implement?
3. What metrics would you monitor?

Use the framework below to structure your response:

In [ ]:
# Exercise 1: Ethical Analysis

analysis_template = {
    'Fairness': {
        'concerns': "[Your answer]",
        'mitigations': "[Your answer]",
        'metrics': "[Your answer]"
    },
    'Transparency': {
        'concerns': "[Your answer]",
        'mitigations': "[Your answer]",
        'metrics': "[Your answer]"
    },
    'Accountability': {
        'concerns': "[Your answer]",
        'mitigations': "[Your answer]",
        'metrics': "[Your answer]"
    },
    'Privacy': {
        'concerns': "[Your answer]",
        'mitigations': "[Your answer]",
        'metrics': "[Your answer]"
    },
    'Beneficence': {
        'concerns': "[Your answer]",
        'mitigations': "[Your answer]",
        'metrics': "[Your answer]"
    }
}

# Example for Fairness principle (you should complete the rest)
example_analysis = {
    'principle': 'Fairness',
    'concerns': 'Age bias: older employees might be flagged as flight risks based on retirement patterns',
    'mitigations': 'Audit model for performance disparities across age groups; use equalized odds metric',
    'monitoring': 'Track true positive rates by age group monthly'
}

print("\nEXERCISE 1: ETHICAL ANALYSIS - TURNOVER PREDICTION MODEL")
print("="*70)
print("\nExample Analysis for Fairness:")
for key, value in example_analysis.items():
    print(f"{key.title()}: {value}")
    
print("\n\nNow complete your analysis for all five principles...")
print("Save your responses in the analysis_template dictionary above.")

## Exercise 2: IRB Determination

Classify each research scenario and specify whether IRB review is needed and at what level.

**Scenario 1**: A software company anonymously surveys 500 current users about product satisfaction. Responses are collected via online form, no personal information is collected, and results will be used internally only.

**Scenario 2**: A university study asks pregnant women about medication use during pregnancy. The researcher will link survey responses to medical records to compare outcomes. This involves gathering new data (surveys) and accessing sensitive medical information.

**Scenario 3**: A data science team analyzes a public dataset of de-identified patient outcomes from 2015-2020 to test whether a new algorithm can predict hospital readmission rates. The dataset was previously published in a journal and contains no identifiable information.

For each scenario, determine: **Does it require IRB? If yes, what level?** Justify your answer.

In [ ]:
# Exercise 2: IRB Determination

scenario_evaluations = {
    'Scenario 1': {
        'requires_irb': "???",
        'review_level': "???",
        'justification': "[Your explanation]"
    },
    'Scenario 2': {
        'requires_irb': "???",
        'review_level': "???",
        'justification': "[Your explanation]"
    },
    'Scenario 3': {
        'requires_irb': "???",
        'review_level': "???",
        'justification': "[Your explanation]"
    }
}

print("\nEXERCISE 2: IRB DETERMINATION")
print("="*70)
print("\nComplete the scenario_evaluations dictionary with your answers.")
print("\nConsider these factors:")
print("  - Does it involve human subjects?")
print("  - Does it have interaction/intervention with subjects?")
print("  - Are personal identifiable data used?")
print("  - Is there minimal risk to subjects?")
print("  - Are data truly de-identified/anonymous?")

## Exercise 3: GDPR Compliance Check

Your company is deploying a new recommendation engine in Germany. It processes:  
- User email addresses
- Purchase history
- Location data
- Browsing behavior

You need to prepare a compliance checklist. Use the privacy by design framework to create a data protection strategy.

**Requirements**:
1. Write GDPR-compliant consent language
2. Specify data minimization strategy
3. Outline retention policies
4. Define access controls
5. Document purpose limitations

In [ ]:
# Exercise 3: GDPR Compliance Strategy

gdpr_compliance_plan = {
    'consent_language': """[Write your GDPR-compliant consent text here]
    
Must include:
    - Freely given option (can refuse)
    - Specific purpose
    - Clear data types
    - How data will be used
    - Right to withdraw
    - Opt-in (not pre-checked)
    """,
    
    'data_minimization': {
        'necessary_fields': "[List only essential data]",
        'optional_fields': "[List nice-to-have data to remove]",
        'justification': "[Why each necessary field is needed]"
    },
    
    'retention_policy': {
        'purchase_history': "[Days/months to retain]",
        'browsing_data': "[Days/months to retain]",
        'email': "[Days/months to retain]",
        'deletion_triggers': "[When to automatically delete]"
    },
    
    'access_controls': {
        'data_scientist_access': "[What can they see?]",
        'product_manager_access': "[What can they see?]",
        'external_vendors': "[Any third party access?]",
        'audit_logging': "[How to track access?]"
    },
    
    'purpose_limitation': {
        'primary_purpose': "[Recommendation engine only]",
        'prohibited_uses': "[List what data cannot be used for]",
        'new_uses': "[How to handle requests for new uses?]"
    }
}

print("\nEXERCISE 3: GDPR COMPLIANCE STRATEGY")
print("="*70)
print("\nComplete the gdpr_compliance_plan dictionary with your detailed strategy.")
print("\nRemember GDPR principles:")
print("  ✓ Data minimization: Collect only what you need")
print("  ✓ Purpose limitation: Use data only for stated purposes")
print("  ✓ Storage limitation: Delete when no longer needed")
print("  ✓ Transparency: Clear communication with users")
print("  ✓ Accountability: Document everything")

## Exercise 4: Bias Detection and Mitigation

You build a model to recommend criminal defendants for early release. Testing reveals:
- Overall accuracy: 82%
- Accuracy for white defendants: 87%
- Accuracy for Black defendants: 71%
- False positive rate (incorrect dangerous rating): 8% white, 22% Black

**Task**:  
1. Identify which fairness criterion this violates
2. Explain the real-world harm
3. Propose three mitigation strategies (pre/in/post-processing)
4. What trade-offs would you accept?

In [ ]:
# Exercise 4: Bias Detection Analysis

bias_analysis = {
    'violated_criterion': "[Which fairness metric is violated?]",
    'explanation': "[Why is this a problem?]",
    
    'real_world_harm': """[Describe concrete harms:
        - Black defendants incorrectly marked as dangerous
        - False positives prevent release for less dangerous individuals
        - Perpetuates racial disparities in criminal justice]""",
    
    'mitigation_strategies': {
        'preprocessing': "[Data collection or feature engineering approaches]",
        'in_processing': "[Model training modifications]",
        'post_processing': "[Prediction adjustment techniques]"
    },
    
    'acceptable_tradeoffs': "[What performance loss is acceptable to achieve fairness?]"
}

print("\nEXERCISE 4: BIAS IN CRIMINAL JUSTICE")
print("="*70)
print("\nThis is a real issue: COMPAS algorithm showed similar bias.")
print("\nAnalyze this scenario in the bias_analysis dictionary.")
print("\nConsider:")
print("  - Demographic parity vs equalized odds trade-off")
print("  - Cost of different types of errors")
print("  - Whether perfect fairness is even possible")
print("  - Who decides the acceptable trade-off?")

## 5. Professional Ethics Codes and EU Guidelines

Multiple professional organizations have established ethics codes for AI and research:

### Major Ethics Codes and Standards

| Organization | Code | Key Focus |
|--------------|------|----------|
| **ACM** | Code of Ethics and Professional Conduct | Software engineering ethics |
| **IEEE** | Ethically Aligned Design | AI and autonomous systems |
| **UNESCO** | Recommendations on AI Ethics | Global perspective |
| **OECD** | AI Principles | Cross-national guidelines |
| **EU** | Ethics Guidelines for Trustworthy AI | GDPR + fairness |

### EU Ethics Guidelines: Seven Key Requirements

The European Commission's guidelines require:

1. **Human Agency and Oversight**: Humans maintain meaningful control
2. **Technical Robustness and Safety**: Systems are secure and reliable
3. **Privacy and Data Governance**: GDPR + data quality
4. **Transparency**: Explainability and communication
5. **Diversity, Non-Discrimination, and Fairness**: Representation and equity
6. **Accountability**: Responsibility mechanisms
7. **Environmental and Societal Well-being**: Broader impact consideration

## Summary and Key Takeaways

### Five Core Ethical Principles

✅ **Fairness**: Systematically audit for bias; track disparate impact metrics

✅ **Transparency**: Explain model decisions to stakeholders; document assumptions

✅ **Accountability**: Assign clear responsibility; maintain audit trails

✅ **Privacy**: Implement privacy by design; enforce GDPR compliance

✅ **Beneficence**: Balance benefits and harms; protect vulnerable populations

### IRB Requirements

✅ IRB review required when research involves **human subjects + intervention/interaction + PII**

✅ Exemption categories exist (secondary analysis of de-identified data)

✅ Three review levels: Exempt → Expedited → Full Board (by risk level)

✅ Different standards apply globally (GDPR in EU, Common Rule in US)

### GDPR Compliance

✅ Applies to any organization processing EEA residents' data

✅ Consent must be freely given, specific, informed, unambiguous, revocable

✅ Privacy by design: minimize, encrypt, access control, retention limits

✅ Violating GDPR costs up to €20M or 4% of global revenue

### Bias Detection and Mitigation

✅ Multiple fairness metrics (demographic parity, equalized odds) capture different aspects

✅ Three stages: pre-processing (data), in-processing (training), post-processing (predictions)

✅ Fairness metrics often conflict; choose the right definition for your use case

✅ Regular auditing and monitoring are essential ongoing practices

### Practical Implementation

✅ Use ethical checklists before deployment (like the examples in Exercises 1-4)

✅ Document all ethical decisions and trade-offs

✅ Build diverse teams; include affected stakeholders in decisions

✅ Plan for monitoring bias and harms post-deployment

✅ Remember: ethical research is not about perfect solutions, but thoughtful choices

## What's Next?

**Module 10: Research Communication** will cover:
- Writing research papers with integrity
- Creating transparent visualizations
- Disclosing limitations and conflicts of interest
- Engaging with policy makers and public audiences

## Additional Resources

### Books
- "Ethics for AI" by Shannon Vallor
- "Fairness and Machine Learning" by Barocas, Hardt, Narayanan (free online)
- "The Ethical Algorithm" by Kearns & Roth

### Tools
- **Fairlearn**: Microsoft's fairness library
- **AIF360**: IBM's algorithmic fairness toolkit
- **Lime/SHAP**: Model explainability tools

### Official Guidelines
- EU Ethics Guidelines for Trustworthy AI
- IEEE Ethically Aligned Design
- ACM Code of Ethics and Professional Conduct
- GDPR Official Text (gdpr-info.eu)

### Case Studies
- COMPAS algorithm: Racial bias in recidivism prediction
- Amazon hiring AI: Gender bias in resume screening
- Facial recognition: Disparate performance across race/gender
- Loan approval systems: Redlining patterns in modern ML

## Self-Assessment

Before moving to Module 10, ensure you can:

- [ ] Explain the five core ethical principles and give examples of each
- [ ] Determine whether research requires IRB review and at what level
- [ ] Identify when GDPR applies and implement compliant consent processes
- [ ] Calculate fairness metrics (demographic parity, equalized odds)
- [ ] Detect bias across demographic groups in ML models
- [ ] Propose bias mitigation strategies at pre/in/post-processing stages
- [ ] Implement privacy by design principles in systems
- [ ] Conduct ethical analysis using frameworks like the five principles
- [ ] Recognize fairness metric trade-offs and make intentional choices
- [ ] Apply professional ethics codes to real research scenarios

If you can confidently check all boxes, you're ready for Module 10! 🎉